In [37]:
from pyspark.sql.types import StructType
from pyspark.sql.types import *
from pyspark.sql import SparkSession
# from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.streaming import DataStreamWriter
from minio import Minio
from datetime import timedelta
from delta.tables import *
import os
from pyspark.sql.window import Window

def minio_session_spark():
    spark = (
        SparkSession.builder
            .master("local[*]")
            .appName("appMinIO")
            ### Config Fields
            .config('spark.sql.debug.maxToStringFields', 5000)
            .config('spark.debug.maxToStringFields', 5000)
            ### Optimize
            .config("delta.autoOptimize.optimizeWrite", "true")
            .config("delta.autoOptimize.autoCompact", "true")
            ### Delta Table
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            ## MinIO
            #.config("spark.hadoop.fs.s3a.endpoint", "http://172.20.0.2:9000")
             .config("spark.hadoop.fs.s3a.endpoint", "minio:9000")

            .config("spark.hadoop.fs.s3a.access.key", "tcc_user")
            .config("spark.hadoop.fs.s3a.secret.key", "Acnmne@a9h!")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
            ## Jars
            .config("spark.jars", "/home/jovyan/work/jars/hadoop-common-3.3.2.jar,\
                                    /home/jovyan/work/jars/hadoop-aws-3.3.2.jar, \
                                    /home/jovyan/work/jars/aws-java-sdk-bundle-1.11.874.jar")
            .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
            .getOrCreate()
    )
    return spark

### BRONZE PLAYERS

In [38]:
spark = minio_session_spark()

# # spark
# print(f"Spark version = {spark.version}")

# # hadoop
# print(f"Hadoop version = {spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

In [41]:
df = (
spark
    .read
    .format('delta')
    .load(f"s3a://gold/tb_lol_bronze_players")
)

#define a janela 
window_spec = (
    Window
    .partitionBy("rank")
    .orderBy(F.col("leaguePoints").desc(),
             F.col("win_percentage").desc())
)
# #Cria % de vitorias e rank dos too players
# df_perc_wins = (
#     df
#     .withColumn("win_percentage", F.col("wins") / ( F.col("wins") + F.col("losses" )))
#     .select("summonerId", "rank","leaguePoints","win_percentage", "wins", "losses")
#     .withColumn("rn", F.row_number().over(window_spec))
# )

In [57]:
#Cria lista dos top 2 players
list_top_players = (
    df
    .filter(( F.col("wins") + F.col("losses") >= 20))
    .withColumn("win_percentage", F.col("wins") / ( F.col("wins") + F.col("losses" )))
    .withColumn("rn", F.row_number().over(window_spec))
    # .select("summonerId", "rank","leaguePoints","win_percentage", "wins", "losses", "rn")
    .select("summonerId")
    .filter(F.col("rn") <= 2)
    .rdd
    .flatMap(lambda x: x)
    .collect()
)

In [58]:
list_top_players

['4gfWwYGaQd3l0WBBSO5dTjfozf4n_2ubqg3CiyegTFrr1w',
 'gTdA9kOKnY3439FvhhTyZpQOGSzAWWYsFj0C5eXysvV2fJk',
 'S6mIk7bnYcth7wff1X4RGklcPUmbqBF9OaN_87eg6aOQHJU',
 'cLNEYXDDA6kq1ntd1VjkfMg9umY1Su8spwTd2ShUanq1DoA',
 'J1su2SOzNTjj5yIOZ_UeYxTnDpjAukgj9DwCJJy1gE1MqzE',
 'mmiqBn1nXXiFlA2-huEcXOL8M_KT8L26Cu9ww5s6XXSgT0E']

#### COLUMNS & TYPES

In [14]:
df.dtypes

[('freshBlood', 'boolean'),
 ('hotStreak', 'boolean'),
 ('inactive', 'boolean'),
 ('leagueId', 'string'),
 ('leaguePoints', 'bigint'),
 ('losses', 'bigint'),
 ('queueType', 'string'),
 ('rank', 'string'),
 ('summonerId', 'string'),
 ('summonerName', 'string'),
 ('tier', 'string'),
 ('veteran', 'boolean'),
 ('wins', 'bigint')]

#### NUMBER OF PLAYERS

In [24]:
df.select(F.countDistinct('summonerId')).collect()[0][0]

229310

#### % NULL VALUES

In [22]:
# Calculate the percentage of null values in each column
total_rows = df.count()
null_percentages = {}

for column in df.columns:
    column_type = dict(df.dtypes)[column]
    if column_type in ['double','float','bigint']:
        null_count = df.filter(F.col(column).isNull() | F.isnan(F.col(column))).count()
    else:
        null_count = df.filter(F.col(column).isNull()).count()
                               
    null_percentage = (null_count / total_rows) * 100
    null_percentages[column] = null_percentage

# Display the results
for column, percentage in null_percentages.items():
    print(f"Column '{column}' has {percentage:.2f}% null values")

Column 'freshBlood' has 0.00% null values
Column 'hotStreak' has 0.00% null values
Column 'inactive' has 0.00% null values
Column 'leagueId' has 0.00% null values
Column 'leaguePoints' has 0.00% null values
Column 'losses' has 0.00% null values
Column 'queueType' has 0.00% null values
Column 'rank' has 0.00% null values
Column 'summonerId' has 0.00% null values
Column 'summonerName' has 0.00% null values
Column 'tier' has 0.00% null values
Column 'veteran' has 0.00% null values
Column 'wins' has 0.00% null values
